In [ ]:
# Generate image patches by landmark

In [1]:
%reload_ext autoreload
%autoreload 2

import cv2

import os
import sys
sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import DataManager

from collections import defaultdict

from matplotlib.path import Path
%matplotlib inline

import pandas as pd
import numpy as np

In [2]:
# stack = 'MD589'
stack = 'MD594'

patch_dir = '/home/yuncong/CSHL_data_patches/'
table_filepath = os.path.join(patch_dir, '%(stack)s_indices_allLandmarks_allSections.h5'%{'stack':stack})
indices_allLandmarks_allSections = pd.read_hdf(table_filepath, 'indices_allLandmarks_allSections')
grid_parameters = pd.read_hdf(table_filepath, 'grid_parameters')

In [ ]:
indices_allLandmarks_allSections

In [ ]:
patch_size, stride, w, h = grid_parameters.tolist()

print 'patch_size', patch_size
print 'stride', stride
print 'w, h', w, h

half_size = patch_size/2
ys, xs = np.meshgrid(np.arange(half_size, h-half_size, stride), np.arange(half_size, w-half_size, stride),
                     indexing='xy')
sample_locations = np.c_[xs.flat, ys.flat]

In [5]:
byLandmark_dir = '/home/yuncong/CSHL_data_patches/%(stack)s_byLandmark/'%{'stack':stack}

In [ ]:
for label in list(indices_allLandmarks_allSections.index) + ['BackG']:
    output_dir = os.path.join(byLandmark_dir, label)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

In [ ]:
for sec in indices_allLandmarks_allSections.keys():
    
    print sec    
    dm = DataManager(stack=stack, section=sec, load_mask=False)
    dm._load_image(['rgb-jpg'])

    for label, indices in dict(indices_allLandmarks_allSections[sec].dropna()).iteritems():
        
        # do not dump bg patches
        if label == 'bg':
            continue
        
        for i in indices:
#         for i in np.random.choice(indices, min(10, len(indices)), replace=False):
            x, y = sample_locations[i]
            patch = dm.image_rgb_jpg[y-half_size:y+half_size, x-half_size:x+half_size]
            
            output_dir = os.path.join(byLandmark_dir, label)
            patch_name = '%(stack)s_%(sec)04d_%(gridp_ind)08d.jpg' % {'stack':stack, 'sec':sec, 'gridp_ind':i}
            cv2.imwrite(os.path.join(output_dir, patch_name), patch[..., ::-1])
            
#             # if is surround, additionally dump to BackG
#             if label.endswith('surround'):
#                 output_dir = os.path.join(byLandmark_dir, 'BackG')
#                 cv2.imwrite(os.path.join(output_dir, patch_name), patch[..., ::-1])

In [ ]:
# Count number of patches each landmark
c = 0
for l in os.listdir(byLandmark_dir):
    n = len(os.listdir(os.path.join(byLandmark_dir, l)))
    print l, n
    c += n
    
print 'total number of patches =', c

In [3]:
tar_dir = os.path.join(patch_dir, stack+'_byLandmark_tarballs')
if not os.path.exists(tar_dir):
    os.makedirs(tar_dir)

In [6]:
for l in os.listdir(byLandmark_dir):
#     print l
    cmd = 'tar -C %(patch_dir)s/%(stack)s_byLandmark -I pigz -cf %(tar_dir)s/%(label)s.tar %(label)s' % \
              {'label': l, 'stack': stack, 'patch_dir': patch_dir,
               'tar_dir': tar_dir}
    print cmd
#     os.system(cmd)

tar -C /home/yuncong/CSHL_data_patches//MD594_byLandmark -I pigz -cf /home/yuncong/CSHL_data_patches/MD594_byLandmark_tarballs/7n_surround.tar 7n_surround
tar -C /home/yuncong/CSHL_data_patches//MD594_byLandmark -I pigz -cf /home/yuncong/CSHL_data_patches/MD594_byLandmark_tarballs/7N.tar 7N
tar -C /home/yuncong/CSHL_data_patches//MD594_byLandmark -I pigz -cf /home/yuncong/CSHL_data_patches/MD594_byLandmark_tarballs/Gr.tar Gr
tar -C /home/yuncong/CSHL_data_patches//MD594_byLandmark -I pigz -cf /home/yuncong/CSHL_data_patches/MD594_byLandmark_tarballs/SuVe.tar SuVe
tar -C /home/yuncong/CSHL_data_patches//MD594_byLandmark -I pigz -cf /home/yuncong/CSHL_data_patches/MD594_byLandmark_tarballs/Pn.tar Pn
tar -C /home/yuncong/CSHL_data_patches//MD594_byLandmark -I pigz -cf /home/yuncong/CSHL_data_patches/MD594_byLandmark_tarballs/LVe_surround.tar LVe_surround
tar -C /home/yuncong/CSHL_data_patches//MD594_byLandmark -I pigz -cf /home/yuncong/CSHL_data_patches/MD594_byLandmark_tarballs/VLL_surro